# Product Amount Forecast by Time

## Data

Info about this data set: https://fred.stlouisfed.org/series/IPN31152N


Units:  Index 2012=100, Not Seasonally Adjusted

Frequency:  Monthly

The industrial production (IP) index measures the real output of all relevant establishments located in the United States, regardless of their ownership, but not those located in U.S. territories.

NAICS = 31152

Source Code: IP.N31152.N

Suggested Citation:
Board of Governors of the Federal Reserve System (US), Industrial Production: Nondurable Goods: Ice cream and frozen dessert [IPN31152N], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/IPN31152N, November 16, 2019.

## Import Libraries

## Indest Data

## Exploratory Data Analysis and Visualization

1. Implement basic steps to see how is your data looks like
2. Change the column name as "Production"
2. Plot your data and see its seasonalty

## Preprocessing of Data

### Train Test Split

### Scaling

## Time Series Generator

## Modelling & Model Performance

### Import related libraries

### Creating Model

### with "tanh" activation function

#### Evaluate on Test Data

#### Inverse Transformations and Comparing

#### Calculation R2 Score and Error Metrics

### with "relu" activation function

#### Evaluate on Test Data

#### Inverse Transformations and Comparing

#### Calculation R2 Score and Error Metrics

## Retrain and Forecasting

Select activation function providing a better score, create your final model with full data, forecast for the next 12 months, and plot this forecast.